In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ampc-dataset-main/ampc/w2.csv
/kaggle/input/ampc-dataset-main/ampc/w4.csv
/kaggle/input/ampc-dataset-main/ampc/w1.csv
/kaggle/input/ampc-dataset-main/ampc/w3.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

In [3]:
file_paths = [f'/kaggle/input/ampc-dataset-main/ampc/w{i}.csv' for i in range(1, 5)]

data = pd.concat([pd.read_csv(f) for f in file_paths], ignore_index=True)

data.to_csv('./combined_data.csv', index = False)

data.head()

,acc_mean_x_right,acc_mean_y_right,acc_mean_z_right,acc_mean_xyz_right,acc_mean_xy_right,acc_mean_yz_right,acc_mean_zx_right,acc_mean_pitch_right,acc_mean_roll_right,acc_std_x_right,...,gyro_max_yz_left,gyro_max_zx_left,gyro_peak_x_left,gyro_peak_y_left,gyro_peak_z_left,gyro_peak_xyz_left,gyro_peak_xy_left,gyro_peak_yz_left,gyro_peak_zx_left,class
0,-0.17330,0.14864,0.98128,1.1065,0.44735,1.06110,1.03320,9.9751,52.281,0.25398,...,137.85,79.286,4,4,3,2,2,2,4,2
1,-0.40618,0.24715,0.79471,1.0178,0.52388,0.86595,0.96693,-30.4210,48.213,0.26456,...,269.08,103.560,3,1,2,2,2,2,1,2
2,-0.49670,0.37167,0.70283,1.0402,0.68213,0.80958,0.96510,-23.0680,52.897,0.35638,...,158.42,114.700,2,3,2,1,1,2,2,2
3,-0.28780,0.15882,0.91688,1.0974,0.50834,1.02760,0.99884,3.2451,31.009,0.29577,...,283.65,120.460,3,2,2,3,4,3,2,2
4,-0.56189,0.36946,0.68668,1.3085,0.91759,1.02180,1.12010,-24.1180,47.579,0.56810,...,199.69,93.039,4,2,2,3,3,3,2,2


In [4]:
data = pd.read_csv("combined_data.csv")  

shuffled_data = data.sample(frac=1, random_state=42).reset_index(drop=True)

shuffled_data.to_csv("all_data.csv", index=False)

In [5]:
shuffled_data = pd.read_csv("all_data.csv")

shuffled_data.head()

,acc_mean_x_right,acc_mean_y_right,acc_mean_z_right,acc_mean_xyz_right,acc_mean_xy_right,acc_mean_yz_right,acc_mean_zx_right,acc_mean_pitch_right,acc_mean_roll_right,acc_std_x_right,...,gyro_max_yz_left,gyro_max_zx_left,gyro_peak_x_left,gyro_peak_y_left,gyro_peak_z_left,gyro_peak_xyz_left,gyro_peak_xy_left,gyro_peak_yz_left,gyro_peak_zx_left,class
0,-0.47189,0.30558,0.831100,1.0034,0.56224,0.88554,0.95576,-40.481,53.233,0.006978,...,1.589,1.0254,9,8,8,8,7,8,9,0
1,0.84220,0.35072,0.308980,1.2641,1.05270,0.74039,1.13660,71.101,32.668,0.446530,...,271.980,154.8600,7,6,4,6,6,6,5,2
2,-0.98024,0.52331,0.061482,1.3013,1.17830,0.74973,1.13200,-56.072,35.152,0.369990,...,50.073,76.1390,4,5,4,4,4,5,4,2
3,-0.20462,0.33777,0.900730,1.1621,0.62003,1.05140,1.03660,-14.699,54.450,0.478460,...,171.610,208.9000,3,5,3,2,2,4,3,2
4,-0.33887,-0.81768,0.443720,1.1962,1.08860,1.01700,0.72398,-12.913,-49.806,0.493840,...,252.480,163.4900,2,4,1,4,5,5,2,2


In [6]:
x = shuffled_data.drop(columns = 'class')
y = shuffled_data['class']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) 

clf = svm.SVC() 
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test) 
accuracy_score = accuracy_score(y_test, y_pred) 

print(accuracy_score)

0.8882201203783319


In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

clf = svm.SVC()
scores = cross_val_score(clf, x, y, cv = 10) # 10-fold cross validation

print (scores)

[0.90283749 0.89423904 0.89251935 0.88392089 0.89595873 0.89595873
 0.88048151 0.89337919 0.90025795 0.87865749]


In [8]:
f"{scores.mean()*100:2f}%"

'89.182104%'

In [9]:
from sklearn.model_selection import GridSearchCV 
  
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, n_jobs = -1) 
  
# fitting the model for grid search 
grid.fit(x_train, y_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits


GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [10]:
optimal_svc = svm.SVC(kernel='rbf', C = grid.best_params_['C'], gamma = grid.best_params_['gamma'])
optimal_svc.fit(x_train, y_train)

SVC(C=10, gamma=0.0001)

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

y_pred_tune = optimal_svc.predict(x_test)
accuracy_new = accuracy_score(y_test, y_pred_tune)
print(accuracy_new)

0.8420750931498997


In [12]:
scores = cross_val_score(optimal_svc, x, y, cv=10, scoring='accuracy', n_jobs=-1)
print(scores)

[0.8383491  0.8460877  0.84092863 0.84264832 0.84522786 0.8460877
 0.84264832 0.84178848 0.84436801 0.83820998]


In [13]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

In [14]:
selector = SelectKBest(f_classif, k=100)

x_selected = selector.fit_transform(x, y)

x_train, x_test, y_train, y_test = train_test_split(x_selected, y, test_size=0.3, random_state=42)

In [15]:
svc_selected = SVC(kernel='rbf', C = grid.best_params_['C'], gamma = grid.best_params_['gamma'])

svc_selected.fit(x_train, y_train)

y_pred_selected = svc_selected.predict(x_test)

accuracy_selected = accuracy_score(y_test, y_pred_selected)

print(f"{accuracy_selected*100:.2f}%")

85.61%


In [16]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

In [17]:
pca = PCA(n_components=10)

x_pca = pca.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x_pca, y, test_size=0.3, random_state=42)

In [18]:
svc_pca = SVC(kernel='rbf', C = grid.best_params_['C'], gamma = grid.best_params_['gamma'])

svc_pca.fit(x_train, y_train)

y_pred_pca = svc_pca.predict(x_test)

accuracy_pca = accuracy_score(y_test, y_pred_pca)

print(f"{accuracy_pca*100:.2f}%")

84.21%


In [19]:
cv_scores = cross_val_score(svc_pca, x_pca, y, cv=10)

accuracy_pca = cv_scores.mean()

print(f"{accuracy_pca * 100:.2f}%")

84.36%


In [20]:
# Import necessary libraries
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [21]:
models = {
    "SGD": SGDClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "MLP": MLPClassifier(random_state=42)
}

for name, model in models.items():
    model.fit(x_train, y_train)
    
    y_pred = model.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    
    print(f"{name}: {test_accuracy * 100:.2f}%")

SGD: 61.13%
Random Forest: 90.37%
MLP: 82.43%


In [22]:
for name, model in models.items():
    cv_scores = cross_val_score(model, x, y, cv=10)
    cv_accuracy = cv_scores.mean()
    
    print(f"10-Fold Cross-Validation Accuracy using {name}: {cv_accuracy * 100:.2f}%")

10-Fold Cross-Validation Accuracy using SGD: 87.83%
10-Fold Cross-Validation Accuracy using Random Forest: 92.54%
10-Fold Cross-Validation Accuracy using MLP: 87.02%
